<a href="https://colab.research.google.com/github/crew-guy/ml-with-tensorflow/blob/main/basics-tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we are going to cover some of the most fundamental concepts of tensors using TensorFlow

* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up Python functions)
* Using GPUs with tensorflow (or TPUs)

## Introduction to tensors

In [ ]:
# Import tensorflow

import tensorflow as tf
print(tf.__version__)


2.9.2


In [ ]:
 # Create a tensor from a tensor like object
 scalar = tf.constant(7)
 print(scalar)

tf.Tensor(7, shape=(), dtype=int32)


In [ ]:
# Check the number of dimensions of a tensor
print(scalar.ndim)  

0


In [ ]:
# Create a vector
vector = tf.constant([10,10])
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [ ]:
# Create a matrix
matrix = tf.constant([[1,2],[3,4]])
    print(matrix, matrix.ndim)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 2


In [ ]:
# Specify the data type with the dtype parameter
matrix2 = tf.constant([
    [1.,2.],
    [3.,4.],
    [5.,6.]
    ], dtype = tf.float16)
print(matrix2, matrix2.ndim)

tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]], shape=(3, 2), dtype=float16) 2


 ## What we've created so far
Scalar: A constant number
<br/>
Vector: A array of numbers representing a magnitude with a direction (eg: wind speed)
<br/>
Matrix: A 2-D array of numbers
<br/>
Tensor: An n-dimensional array of numbers


## Creating tensors withh `tf.Variable`

In [ ]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Values inside a changeable_tensor (created with tf.Variable) can be reassigned whereas the same cannot be done for unchangeable_tensor

changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

🔑 Note : Rarely in practice, we decide, whether to use tf.constant() or tf.Variable, as Tensorflow does this for use. However, if in doubt, use tf.constant and change it later

## Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers



In [ ]:
# Create 2 random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [ ]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

## Shuffle the order of elements in a tensor

Valuable when you want to shuffle your data so that inherent order does not affect learning

> Rule : "If want to make our tensor shuffling reproducible, then we have to specify global & local tensors"

In [ ]:
# Shuffling a tensor
not_shuffled = tf.Variable([[10,7],[6,5],[3,2]])

# Using random shuffle - shuffling happens along the outermost dimension, i.e. dimension-0
shuffled = tf.random.shuffle(not_shuffled)

shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  2],
       [10,  7],
       [ 6,  5]], dtype=int32)>

In [ ]:
# Setting a seed for shuffling will allow reproducibity - https://www.tensorflow.org/api_docs/python/tf/random/set_seed
# Notion notes for understanding seed - https://www.notion.so/techsoc/Understanding-Seeds-29579a6f8b6c4f4a828cfef8629f09d6

# Setting "global seed"
tf.random.set_seed(22)


# Setting operation-level aka "local seed"
# Local seeds override global seeds
tf.random.shuffle(not_shuffled, seed= 21)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 6,  5],
       [10,  7],
       [ 3,  2]], dtype=int32)>

## Other ways of creating a tensor

In [ ]:
# Using tf.ones & tf.zeros

# We can pass the shape of the tensor as an array or a tuple

ones_t = tf.ones([3,4], dtype=tf.float16)
zeros_t = tf.zeros(shape=(3,3,2))

ones_t, zeros_t

(<tf.Tensor: shape=(3, 4), dtype=float16, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float16)>,
 <tf.Tensor: shape=(3, 3, 2), dtype=float32, numpy=
 array([[[0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.]]], dtype=float32)>)

In [ ]:
# Creating tensor from NumPy array
# Only difference between numpy array & tensors is that tensor computations can be run on a GPU (much faster)

import numpy as np
numpy_A = np.random.randint(-4, 8, size=(3,4))

print(numpy_A)

tensor_A = tf.constant(numpy_A, shape=(2,3,2))


print(tensor_A)

[[ 3 -2 -3  5]
 [-3  6  5  7]
 [ 2 -2  3 -4]]
tf.Tensor(
[[[ 3 -2]
  [-3  5]
  [-3  6]]

 [[ 5  7]
  [ 2 -2]
  [ 3 -4]]], shape=(2, 3, 2), dtype=int64)


# Getting information from tensors

* Shape - The length (number of elements) of each of the dimensions of the tensor - `tensor1.shape`
* Rank - The number of tensor dimensions - `tensor1.ndim`
* Size - Total number of elements in the tensor - `tf.size(tensor1)`
* Axis/Dimension - A particular dimension of the tensor - `tensor1[AXIS_NUMBER]`

In [ ]:
tensor_1 = tf.ones([2,3,4,5], dtype = tf.float16)

tensor_1, tensor_1.shape, tf.size(tensor_1), tensor_1.ndim

(<tf.Tensor: shape=(2, 3, 4, 5), dtype=float16, numpy=
 array([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],
 
 
        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]], dtype=float16)>,
 TensorShape([2, 3, 4, 5]),
 <tf.Tensor: shape=(), dtype=int32, numpy=120>,
 4)

In [ ]:
# Get various attributes of our tensor - "pretty print method" - really cool snippet to actually get imp metadata of a tensor
print("Datatype of every element:", tensor_1.dtype)
print("Shape of the tensor:", tensor_1.shape)
print("Number of dimensions of the tensor:", tensor_1.ndim)
print("Elements along the 0 axis:", tensor_1.shape[0])
print("Elements along the last axis:", tensor_1.shape[-1])
print("Total number of elements in the tensor", tf.size(tensor_1))
print("Total number of elements in the tensor", tf.size(tensor_1).numpy())

Datatype of every element: <dtype: 'float16'>
Shape of the tensor: (2, 3, 4, 5)
Number of dimensions of the tensor: 4
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in the tensor tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in the tensor 120


## Indexing & expanding tensors

Tensors can be indexed just like Python lists

In [ ]:
# Get the 1st 2 elements of each dimension

first_2_elems = tensor_1[:2,:2,:2,:2]

# Get all elements, except the final one, from each dimension 

except_last_dim = tensor_1[:-1,:-1,:-1,:-1]

# Get the 1st element from each dimension from each each index, except for the final one

a = tensor_1[:1,:1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float16, numpy=array([[[[1., 1., 1., 1., 1.]]]], dtype=float16)>

In [ ]:
# For rank 2 tensors


rank_2_tensor = tf.constant([[10,7], [32,3]])
# Get the last item of each row of a rank 2 tensor

k = rank_2_tensor[:,-1]

k

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 3], dtype=int32)>

In [ ]:
# Add in extra dimension to our rank-2 tensor
# The "..." notation helps specify all axes of the tensor we are adding an extra dimension to
rank_3_tensor = rank_2_tensor[..., tf.newaxis]

# Alternative method - manually specify all axes for the tensor we are mutating
# rank_3_tensor = rank_2_tensor[:,:, tf.newaxis]

rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[32],
        [ 3]]], dtype=int32)>

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand on the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[32],
        [ 3]]], dtype=int32)>

In [ ]:
# Expanding along 0-th axis
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [32,  3]]], dtype=int32)>

## Manipulating tensors (tensor operations)


**Basic operations** - `+`, `-`, `/`, `*`

In [ ]:
# Original tensor is unchanged

tensor1 = tf.constant([[10,7],[3,4]])
tensor2 = tensor1+10
tensor3 = tensor1*10

tensor2, tensor3

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[20, 17],
        [13, 14]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  70],
        [ 30,  40]], dtype=int32)>)

In [ ]:
# We can use the tensorflow function as well
tf.subtract(tensor1, 3)

# Alternative
# tf.math.subtract(tensor1,3)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[7, 4],
       [0, 1]], dtype=int32)>

**Matrix Multiplication**

 In machine learning, matrix multiplication is one of the the common tensor operations

 Use this online tool - matrixmultiplication.xyz to better understand how matrix multiplication works

In [ ]:
# Matrix multiplication in tensorflow

# Can use the "tf.linalg.matmul" or its short form - "tf.matmul" for multiplying 2 matrices in tf
mt_1 = tf.constant([[[1,2,5], [7,2,1],[3,3,3]]])
mt_2 = tf.constant([[3,5], [6,7], [1,8]])


mt_mult = tf.matmul(mt_1, mt_2)

mt_mult

<tf.Tensor: shape=(1, 3, 2), dtype=int32, numpy=
array([[[20, 59],
        [34, 57],
        [30, 60]]], dtype=int32)>

In [ ]:
# Matrix multiplication using Python operator - "@"

mt_1@mt_2

<tf.Tensor: shape=(1, 3, 2), dtype=int32, numpy=
array([[[20, 59],
        [34, 57],
        [30, 60]]], dtype=int32)>

In [54]:
### Changing the shape of a tensor
mt_reshaped = tf.reshape(mt_1, shape=(3,3))
mt_1@mt_reshaped

<tf.Tensor: shape=(1, 3, 3), dtype=int32, numpy=
array([[[30, 21, 22],
        [24, 21, 40],
        [33, 21, 27]]], dtype=int32)>

In [55]:
mt_reshaped.shape, mt_1.shape

(TensorShape([3, 3]), TensorShape([1, 3, 3]))

In [57]:
# Transpose
# Difference between transpose & reshape is that transpose actually switches row elems with column elements whereas
# reshape feels to just collate all elements in a list & then puts them one by one into the new form
# Thus, transpose is flipping the axes whereas reshaping is just reshuffling

X = tf.constant([[1,2],[3,4],[5,6]], dtype = tf.float16)
X, tf.transpose(X), tf.reshape(X, shape=(2,3))


(<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
 array([[1., 2.],
        [3., 4.],
        [5., 6.]], dtype=float16)>,
 <tf.Tensor: shape=(2, 3), dtype=float16, numpy=
 array([[1., 3., 5.],
        [2., 4., 6.]], dtype=float16)>,
 <tf.Tensor: shape=(2, 3), dtype=float16, numpy=
 array([[1., 2., 3.],
        [4., 5., 6.]], dtype=float16)>)

**The dot product**


Matrix multiplication is also referred to as the dot product

* `tf.matmul()`
* `tf.tensordot()`
* `@` operator

In [61]:
Y = tf.constant([[1,2], [3,4]], dtype=tf.float16)

Y.shape, X.shape, tf.transpose(X).shape

(TensorShape([2, 2]), TensorShape([3, 2]), TensorShape([2, 3]))

In [62]:
# Refer docs of tensordot for reference - https://www.tensorflow.org/api_docs/python/tf/tensordot
# axes = 1 refers to matrix multiplication
tf.tensordot(Y, tf.transpose(X), axes =1)

<tf.Tensor: shape=(2, 3), dtype=float16, numpy=
array([[ 5., 11., 17.],
       [11., 25., 39.]], dtype=float16)>

In [65]:
# At every step, log the shape to better understand what is getting multiplied

# Perform matrix multiplication using transpose
X.shape, Y.shape

matmul_1 = tf.matmul(Y, tf.transpose(X))

# Perform matrix multiplication using reshape
matmul_2 = tf.matmul(tf.reshape(X, shape=(3,2)),Y)

matmul_1, matmul_2

(<tf.Tensor: shape=(2, 3), dtype=float16, numpy=
 array([[ 5., 11., 17.],
        [11., 25., 39.]], dtype=float16)>,
 <tf.Tensor: shape=(3, 2), dtype=float16, numpy=
 array([[ 7., 10.],
        [15., 22.],
        [23., 34.]], dtype=float16)>)

## Change the data type of a tensor

In [70]:
B = tf.constant(7.3)
C = tf.constant(2)

# Change from float32 to float16 (reduced precision)

D = tf.cast(B, dtype=tf.float16)

B.dtype, C.dtype, D.dtype

(tf.float32, tf.int32, tf.float16)

## Aggregating tensors


Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [71]:
# Get the absolute value
E = tf.constant([-7,-10])

tf.abs(E)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Aggregation functions
* mean
* minimum
* maximum
* sum

In [75]:
F = tf.constant(np.random.randint(-6, 8, size=(2,3,4)))
F

<tf.Tensor: shape=(2, 3, 4), dtype=int64, numpy=
array([[[ 2,  5,  7,  5],
        [ 5,  6, -5, -4],
        [-5,  3,  4, -5]],

       [[ 0, -1, -5,  2],
        [ 6,  0,  4, -5],
        [-2,  4, -3,  0]]])>

In [90]:
max_along_y = tf.math.reduce_max(F, axis=1)
mean_along_z = tf.reduce_mean(F, axis=0)
min_along_x = tf.reduce_min(F, axis=2)
sum_along_x = tf.reduce_sum(F, axis=2)


# 🔴 Variance and standard deviation can only be calculated if dtype of tensor's elements is float 
# Also, in variance & std. dev. we don't have the direct aliasing available, so we have to go through tf.math.FUNCTION approach
F = tf.cast(F,dtype = tf.float16)
var_along_y = tf.math.reduce_variance(F, axis=1)
std_along_z = tf.math.reduce_std(F, axis=0)


print("Min along x:","\n", min_along_x,"\n")
print("Sum along x:","\n", sum_along_x,"\n")
print("Max along y:","\n",max_along_y, "\n")
print("Mean along z:","\n",mean_along_z,"\n")
print("Variance along y", "\n", var_along_y,"\n")
print("Std dev along y", "\n", std_along_z,"\n")

Min along x: 
 tf.Tensor(
[[ 2 -5 -5]
 [-5 -5 -3]], shape=(2, 3), dtype=int64) 

Sum along x: 
 tf.Tensor(
[[19  2 -3]
 [-4  5 -1]], shape=(2, 3), dtype=int64) 

Max along y: 
 tf.Tensor(
[[5 6 7 5]
 [6 4 4 2]], shape=(2, 4), dtype=int64) 

Mean along z: 
 tf.Tensor(
[[ 1  2  1  3]
 [ 5  3  0 -4]
 [-3  3  0 -2]], shape=(3, 4), dtype=int64) 

Variance along y 
 tf.Tensor(
[[17.56   1.555 26.    20.23 ]
 [11.56   4.668 14.88   8.664]], shape=(2, 4), dtype=float16) 

Std dev along y 
 tf.Tensor(
[[1.  3.  6.  1.5]
 [0.5 3.  4.5 0.5]
 [1.5 0.5 3.5 2.5]], shape=(3, 4), dtype=float16) 



**Find the positional max & min of a tensor**

Returns the "position" (aka the "index") of the dimension, where the maximum value occurs

In [135]:
argmin_along_x = tf.math.argmin(F, axis=2)
argmax_along_y = tf.argmax(F, axis=1)

argmin_along_x, argmax_along_y


# Checking that the argmax does indeed have the maximum value, using a simple vector
np_A = np.random.randint(1,100, size=(1,100))
A = tf.constant(np_A)
argmax_index = tf.argmax(A, axis=1).numpy()[0]
value_at_argmax_index = A[0, argmax_index]
max_value = tf.reduce_max(A, axis=1)
value_at_argmax_index.numpy() == max_value.numpy()[0]

True

In [140]:
## Squeezing a tensor (removing all single dimensions)
tf.random.set_seed(32)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G_squeezed = tf.squeeze(G)
G.shape, G_squeezed.shape

(TensorShape([1, 1, 1, 1, 50]), TensorShape([50]))

## One-hot encoding tensors

In [141]:
some_list = [0,1,2,3]


hot_list = tf.one_hot(some_list, depth=3)
custom_hot_list = tf.one_hot(some_list, depth=3, on_value="hot", off_value="not_hot")

custom_hot_list, hot_list

<tf.Tensor: shape=(4, 3), dtype=string, numpy=
array([[b'hot', b'not_hot', b'not_hot'],
       [b'not_hot', b'hot', b'not_hot'],
       [b'not_hot', b'not_hot', b'hot'],
       [b'not_hot', b'not_hot', b'not_hot']], dtype=object)>

In [144]:
### Squaring, log & square root

test_tens = tf.constant(tf.random.uniform(shape=[20]))

squared = tf.square(test_tens)
square_rooted = tf.sqrt(tf.cast(test_tens, dtype=tf.float16))
logged = tf.math.log(tf.cast(test_tens, dtype=tf.float16))

squared, square_rooted, logged


(<tf.Tensor: shape=(20,), dtype=float32, numpy=
 array([0.05311766, 0.3849961 , 0.03988442, 0.36892167, 0.09287407,
        0.0576881 , 0.06397801, 0.04706929, 0.0845518 , 0.00797201,
        0.03328406, 0.05310304, 0.00126246, 0.58098394, 0.09871485,
        0.297688  , 0.82422805, 0.02037134, 0.5831851 , 0.9188174 ],
       dtype=float32)>, <tf.Tensor: shape=(20,), dtype=float16, numpy=
 array([0.48  , 0.7876, 0.4468, 0.7793, 0.552 , 0.4902, 0.503 , 0.4658,
        0.539 , 0.2988, 0.4272, 0.48  , 0.1885, 0.873 , 0.5605, 0.7383,
        0.9526, 0.3777, 0.874 , 0.979 ], dtype=float16)>, <tf.Tensor: shape=(20,), dtype=float16, numpy=
 array([-1.468 , -0.477 , -1.611 , -0.4985, -1.188 , -1.426 , -1.375 ,
        -1.528 , -1.235 , -2.416 , -1.701 , -1.468 , -3.338 , -0.2715,
        -1.157 , -0.6064, -0.0968, -1.947 , -0.2695, -0.0424],
       dtype=float16)>)

## Tensors & NumPy

Tensorflow interacts beautifully (interoperability) with NumPy arrays

🔑 Tensor can be run much better on a GPU or TPU (for faster numerical processing)

In [145]:
 # Create a tensor directly from a numpy array

 tensor = tf.constant(np.array([1,2,3,4]))

 # Convert tensor into numpy array

 np_array = np.array(tensor)

 tensor, np_array, type(np.array(tensor))

(<tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 2, 3, 4])>,
 array([1, 2, 3, 4]),
 numpy.ndarray)

In [147]:
# The "default types" of each are slightly different
numpy_J = tf.constant(np.array([1.1])) # Creating tensor from np array -> dtype is float64
tensor_J = tf.constant([1.1]) # Creating tensor from python list -> dtype is float32

numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

## Access to GPUs

In [2]:
import tensorflow as tf
tf.config.list_physical_devices()

tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Get info on which type of GPU are we using
 !nvidia-smi

Fri Nov 18 11:39:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
🔑 Note: If you have access to a CUDA enabled GPU, tf will use it whenever possible